In [120]:
%load_ext autoreload
%autoreload 2

from denoised_dataset import HaeaVocab, DenoisingDataset
import torch

vocab = HaeaVocab(torch.zeros(1000, 300, 100), torch.zeros(1000, 300, 100), max_len=20)

print(len(vocab))

dataset = DenoisingDataset(
    vocab,
    127,
    mask=0.3,
    mask_random=0,
    insert=0,
    rotate=0,
    permute_sentences=0,
    replace_length=-1,
    mask_length='span-poisson',
    poisson_lambda=3
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1004


In [169]:
print(dataset.dataset[0])

tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23,  2])


In [168]:
data = dataset[0]
print(data['source'].shape, data['source'])


torch.Size([24]) tensor([ 1,  4,  3,  3,  3,  3,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21,  3,  3,  3,  2])


In [20]:
data = torch.zeros(10, 10)
print(data[0].unsqueeze(0).repeat_interleave(5, dim=0).shape)

torch.Size([5, 10])
